# VQE DIY:
Consider a Hermitian matrix as:
\begin{equation}
\begin{split}
M&= \begin{bmatrix}
-0.2524859
 &0.18121
 \\ 
0.18121
 & -1.8318639
\end{bmatrix}\\&=-1.0421749I+ 0.789689Z+ 0.181210X.
\end{split}
\end{equation}

If we operate $R_y(\theta)$  on  $|0\rangle$ :

\begin{equation}
R_y(\theta)|0\rangle=\text{cos}(\theta/2)|0\rangle+\text{sin}(\theta/2)|1\rangle.
\end{equation}
The lowest eigenvalue($E_0$) can be consider as:
\begin{equation}
E_0=\min_\theta\langle0|R_y(\theta)^\dagger HR_y(\theta)|0\rangle=\min_\theta E(\theta)
\end{equation}


In [1]:
import numpy as np 
from qiskit import( QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer)

### Calculating $E(\theta)$ on QC

The expectation value of $Z$ is as
\begin{equation}
\langle Z\rangle=\mathbb{P}_0-\mathbb{P}_1,
\end{equation}
where $(\mathbb{P}_0,\mathbb{P}_1)$ is the probability distribution after the measurement.

If we want to measure is not on Z-axis, we can rotate the bloch vector to change the measure axis via the single-qubit gate.

The expectation value of $X$ is as
\begin{equation}
\langle HZH\rangle=\mathbb{P}_0-\mathbb{P}_1.
\end{equation}


In [2]:
shot=10000

def M(x0):
    theta=x0[0]
    Energy=-1.0421749
    ####caculate the expectation value of 0.789689𝑍
    qr = QuantumRegister(1) # initiate the circuit
    cr = ClassicalRegister(1) #
    Cix= QuantumCircuit(qr, cr) #
    Cix.ry(theta,0) #the ansatz 
    Cix.measure(0,0)       
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(Cix, simulator, shots=shot) 
    result = job.result()#collect the data
    counts = result.get_counts(Cix)
    temp=0
    for n in range (0,len(counts.values())):
        if list(counts.keys())[n]=='0':
            temp+=list(counts.values())[n]/shot
        elif list(counts.keys())[n]=='1':
            temp-=list(counts.values())[n]/shot
    #temp is the expectation value of Z
    Energy+=temp*0.789689
    
    ####caculate the expectation value of 0.181210𝑋
    qr = QuantumRegister(1)
    cr = ClassicalRegister(1)
    Cix= QuantumCircuit(qr, cr)
    Cix.ry(theta,0)
    Cix.h(0)
    Cix.measure(0,0)       
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(Cix, simulator, shots=shot) 
    result = job.result()
    counts = result.get_counts(Cix)
    temp=0
    for n in range (0,len(counts.values())):
        if list(counts.keys())[n]=='0':
            temp+=list(counts.values())[n]/shot
        elif list(counts.keys())[n]=='1':
            temp-=list(counts.values())[n]/shot
    #temp is the expectation value of X    
    Energy+=temp*0.181210
    return Energy

 ### Using the classical optimization algorithm
 scipy.optimize.minimize:
 https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize


In [3]:
from scipy.optimize import minimize
sol = minimize(M,[0],method='COBYLA',options={'rhobeg': 1, 'maxiter': 100, 'disp': False, 'catol': 0.0001})
## classical optimization algorithm‘Nelder-Mead’,‘Powell’,‘L-BFGS-B’,‘COBYLA’ ,‘SLSQP’.........

### The solution of VQE

In [4]:
sol.fun

-1.8526142914000001

In [5]:
import scipy.linalg as la
H= np.array([[-0.2524859,0.18121],[0.18121,-1.8318639]])
exacteigenvalue = la.eig(H)
exacteigenvalue[0][1]

(-1.8523883168359594+0j)

 ### Error rate

In [6]:
error=(sol.fun-exacteigenvalue[0][1])/exacteigenvalue[0][1]
error

(0.00012199092489781242-0j)